In [1]:
# !pip install pysrt
# !pip install scenedetect[opencv] --upgrade
# !pip install openai

In [2]:
video_path = "Episode 001.mp4"
transcript_path = "TVF Tripling S01E01 - 'Toh Chalein...'.srt"

In [3]:
import pysrt
import pandas as pd
import re

# load the subtitle file
subs = pysrt.open(transcript_path)

def srt_to_df(filepath):
    with open(filepath, 'r') as f:
        content = f.read()
        pattern = r'(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\s((?:(?!\d{2}:\d{2}:\d{2},\d{3}).)*?)\n\n'
        matches = re.findall(pattern, content, re.DOTALL)
        
        df = pd.DataFrame(matches, columns=['start_time', 'end_time', 'dialogue'])
        df['dialogue'] = df['dialogue'].str.replace('\n', ' ')  # replace newlines in dialogue with spaces
        
        return pd.DataFrame(df)

# print each line of dialogue
# for sub in subs:
#     print(f'Start time: {sub.start}\nEnd time: {sub.end}\nText: {sub.text}\n\n')

transcript_df = srt_to_df(transcript_path)
transcript_df

transcript_df.to_csv('{}_transcript.csv'.format(video_path.split('.')[0]))

In [4]:
from scenedetect import detect, ContentDetector
from __future__ import print_function
import os
import scenedetect
import pandas as pd

# Standard PySceneDetect imports:
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager
# For caching detection metrics and saving/loading to a stats file
from scenedetect.stats_manager import StatsManager

# For content-aware scene detection:
from scenedetect.detectors.content_detector import ContentDetector

scene_list = detect(video_path, ContentDetector())
scene_list

columns = ["Scene Number", "Starting Frame Time", "Starting Frame Number", "Ending Frame Time", "Ending Frame Number", "Duration of the Scene"]

# Initialize an empty dataframe
scene_df = pd.DataFrame(columns=columns)

# Iterate over each scene in the list
for i, scene in enumerate(scene_list):
    start_time = scene[0].get_timecode()
    start_frame = scene[0].get_frames()
    #fps = scene[0].get_fps()

    end_time = scene[1].get_timecode()
    end_frame = scene[1].get_frames()

    # Add a new row to the dataframe
    scene_df.loc[i] = [i+1, start_time, start_frame, end_time, end_frame, pd.to_datetime(end_time) - pd.to_datetime(start_time)]

scene_df.sort_values(by='Duration of the Scene', ascending = False).head(20)
scene_df

scene_df.to_csv('{}_shots.csv'.format(video_path.split('.')[0]))

In [6]:
display(transcript_df.head(5))
display(scene_df.head(5))

,start_time,end_time,dialogue
0,"00:00:20,160","00:00:22,168",What comes to your mind when you hear the words
1,"00:00:22,168","00:00:23,287",Road Trip?
2,"00:00:24,271","00:00:25,652",Hangover
3,"00:00:25,668","00:00:27,471",Zindagi Na Milegi Dobara
4,"00:00:27,745","00:00:30,341",Dil Chahta Hai


,Scene Number,Starting Frame Time,Starting Frame Number,Ending Frame Time,Ending Frame Number,Duration of the Scene
0,1,00:00:00.000,0,00:00:05.401,135,0 days 00:00:05.401000
1,2,00:00:05.401,135,00:00:35.364,884,0 days 00:00:29.963000
2,3,00:00:35.364,884,00:00:36.924,923,0 days 00:00:01.560000
3,4,00:00:36.924,923,00:00:37.685,942,0 days 00:00:00.761000
4,5,00:00:37.685,942,00:00:41.645,1041,0 days 00:00:03.960000


In [7]:
# Convert to datetime
transcript_df['start_time'] = pd.to_datetime(transcript_df['start_time'])
transcript_df['end_time'] = pd.to_datetime(transcript_df['end_time'])
scene_df['Starting Frame Time'] = pd.to_datetime(scene_df['Starting Frame Time'])
scene_df['Ending Frame Time'] = pd.to_datetime(scene_df['Ending Frame Time'])

# Assign a constant key to each dataframe and merge
transcript_df['key'] = 0
scene_df['key'] = 0
merged_df = pd.merge(transcript_df, scene_df, on='key')

# Filter rows
final_df = merged_df[(merged_df['Starting Frame Time'] <= merged_df['start_time']) & 
                     (merged_df['Ending Frame Time'] >= merged_df['end_time'])]

# Drop the key column
final_df = final_df.drop('key', axis=1)

display(final_df)

,start_time,end_time,dialogue,Scene Number,Starting Frame Time,Starting Frame Number,Ending Frame Time,Ending Frame Number,Duration of the Scene
1,2023-07-08 00:00:20.160,2023-07-08 00:00:22.168,What comes to your mind when you hear the words,2,2023-07-08 00:00:05.401,135,2023-07-08 00:00:35.364,884,0 days 00:00:29.963000
295,2023-07-08 00:00:22.168,2023-07-08 00:00:23.287,Road Trip?,2,2023-07-08 00:00:05.401,135,2023-07-08 00:00:35.364,884,0 days 00:00:29.963000
589,2023-07-08 00:00:24.271,2023-07-08 00:00:25.652,Hangover,2,2023-07-08 00:00:05.401,135,2023-07-08 00:00:35.364,884,0 days 00:00:29.963000
883,2023-07-08 00:00:25.668,2023-07-08 00:00:27.471,Zindagi Na Milegi Dobara,2,2023-07-08 00:00:05.401,135,2023-07-08 00:00:35.364,884,0 days 00:00:29.963000
1177,2023-07-08 00:00:27.745,2023-07-08 00:00:30.341,Dil Chahta Hai,2,2023-07-08 00:00:05.401,135,2023-07-08 00:00:35.364,884,0 days 00:00:29.963000
...,...,...,...,...,...,...,...,...,...
95230,2023-07-08 00:19:42.249,2023-07-08 00:19:44.510,What is all this? These flowers? All this deco...,269,2023-07-08 00:19:40.223,29502,2023-07-08 00:19:52.865,29818,0 days 00:00:12.642000
95524,2023-07-08 00:19:44.535,2023-07-08 00:19:45.954,I just said it,269,2023-07-08 00:19:40.223,29502,2023-07-08 00:19:52.865,29818,0 days 00:00:12.642000
95818,2023-07-08 00:19:45.979,2023-07-08 00:19:48.161,Make these people quiet. It was a small lie.,269,2023-07-08 00:19:40.223,29502,2023-07-08 00:19:52.865,29818,0 days 00:00:12.642000
96112,2023-07-08 00:19:48.193,2023-07-08 00:19:50.260,Pranav went and told everybody.,269,2023-07-08 00:19:40.223,29502,2023-07-08 00:19:52.865,29818,0 days 00:00:12.642000
